# Problem 0: Gene Expression Analysis with Principal Component Analysis

This problem was inspired by a [Kaggle problem](https://www.kaggle.com/crawford/principle-component-analysis-gene-expression). This problem is an application of the material in Notebook 15, but based on a slightly different description and procedure.

You will carry out a PCA on gene expression data from patients with two different types of leukemia. And then you will implement kmeans clustering method on both the original gene set and the gene set reconstructed from PCA, comparing the classification accuracy bewteen two gene sets. Based on the final results, you can draw your own conclusion whether PCA is effective for this application of distinguishing leukemia patients.

## Description of data set
This dataset comes from a proof-of-concept study published in 1999 by Golub, et al. It showed how new cases of cancer could be classified by gene expression monitoring (via DNA microarray) and thereby provided a general approach for identifying new cancer classes and assigning tumors to known classes. These data were used to classify patients with acute myeloid leukemia (AML) and acute lymphoblastic leukemia (ALL).

> **Reference.** T.R. Golub, D.K. Slonim, P. Tamayo, C. Huard, M. Gaasenbeek, J.P. Mesirov, H. Coller, M. Loh, J.R. Downing, M.A. Caligiuri, C.D. Bloomfield, and E.S. Lander. "[Molecular Classification of Cancer: Class Discovery and Class Prediction by Gene Expression](https://www.ncbi.nlm.nih.gov/pubmed/10521349)." _Science_ 286:531-537, 1999.

In [2]:
import os
import pandas as pd
import numpy as np
import scipy as sp
from IPython.display import display, Image

In [3]:
DATA_PATH = "../resource/asnlib/publicdata/"

# Input train dataset
with open(DATA_PATH + "data_set_ALL_AML_train.csv") as fp:
    trainfile = set(fp.read().splitlines())

# Input test dataset
with open(DATA_PATH + "data_set_ALL_AML_independent.csv") as fp:
    testfile = set(fp.read().splitlines())

# Input y label dataset
with open(DATA_PATH + "actual.csv") as fp:
    labels = set(fp.read().splitlines())

assert len(trainfile)==7131 ,  "The train file is corrupted!"
assert len(testfile)==7131, "The test file is corrupted!"
assert len(labels)==74, "The label file is corrupted!"

# Loading the files into a pandas dataframe for you
X_train = pd.read_csv(DATA_PATH + "data_set_ALL_AML_train.csv")
X_test = pd.read_csv(DATA_PATH + "data_set_ALL_AML_independent.csv")
y = pd.read_csv(DATA_PATH + "actual.csv")

print("X_train has {} rows and {} columns: ========>".format(X_train.shape[0], X_train.shape[1]))
display(X_train.head())
print('\n')
print("X_test has {} rows and {} columns: ========>".format(X_test.shape[0], X_test.shape[1]))
display(X_test.head())
print('\n')
print("y has {} rows and {} columns: ========>".format(y.shape[0], y.shape[1]))
display(y.head())

X_train has 7129 rows and 78 columns: ========>


,Gene Description,Gene Accession Number,1,call,2,call.1,3,call.2,4,call.3,...,29,call.33,30,call.34,31,call.35,32,call.36,33,call.37
0,AFFX-BioB-5_at (endogenous control),AFFX-BioB-5_at,-214,A,-139,A,-76,A,-135,A,...,15,A,-318,A,-32,A,-124,A,-135,A
1,AFFX-BioB-M_at (endogenous control),AFFX-BioB-M_at,-153,A,-73,A,-49,A,-114,A,...,-114,A,-192,A,-49,A,-79,A,-186,A
2,AFFX-BioB-3_at (endogenous control),AFFX-BioB-3_at,-58,A,-1,A,-307,A,265,A,...,2,A,-95,A,49,A,-37,A,-70,A
3,AFFX-BioC-5_at (endogenous control),AFFX-BioC-5_at,88,A,283,A,309,A,12,A,...,193,A,312,A,230,P,330,A,337,A
4,AFFX-BioC-3_at (endogenous control),AFFX-BioC-3_at,-295,A,-264,A,-376,A,-419,A,...,-51,A,-139,A,-367,A,-188,A,-407,A




X_test has 7129 rows and 70 columns: ========>


,Gene Description,Gene Accession Number,39,call,40,call.1,42,call.2,47,call.3,...,65,call.29,66,call.30,63,call.31,64,call.32,62,call.33
0,AFFX-BioB-5_at (endogenous control),AFFX-BioB-5_at,-342,A,-87,A,22,A,-243,A,...,-62,A,-58,A,-161,A,-48,A,-176,A
1,AFFX-BioB-M_at (endogenous control),AFFX-BioB-M_at,-200,A,-248,A,-153,A,-218,A,...,-198,A,-217,A,-215,A,-531,A,-284,A
2,AFFX-BioB-3_at (endogenous control),AFFX-BioB-3_at,41,A,262,A,17,A,-163,A,...,-5,A,63,A,-46,A,-124,A,-81,A
3,AFFX-BioC-5_at (endogenous control),AFFX-BioC-5_at,328,A,295,A,276,A,182,A,...,141,A,95,A,146,A,431,A,9,A
4,AFFX-BioC-3_at (endogenous control),AFFX-BioC-3_at,-224,A,-226,A,-211,A,-289,A,...,-256,A,-191,A,-172,A,-496,A,-294,A




y has 72 rows and 2 columns: ========>


,patient,cancer
0,1,ALL
1,2,ALL
2,3,ALL
3,4,ALL
4,5,ALL


## Data Cleaning
**Exercise 0** (`clean_data`: 2.5 points).

First you need to do some data cleaning using pandas.

As you can seen from the previous cell, there are 3 data frames: `X_train`, `X_test` and `y`.

The `X_train` data frame has 7129 rows and 78 columns. Each row is a gene. The first two columns `'Gene Description'` and `"Gene Accession Number"` encode some basic information for each gene. We will be interested primarily in so-called genes corresponding to [mRNA expression values](https://www.biosyn.com/tew/The-role-of-mRNA-in-Gene-Expression.aspx), and we'll explain below how to find those. The rest of the columns hold patient information. The columns whose names consist only of an integer (such as "1", "2", "3") store patient ids. The vaules in these columns are the gene expression values. The columns with "call" in the name are the columns to measure the quality of the detection call. In this problem, you will ignore "call" columns.

`X_test` has similar format as `X_train`. The row labels are the same but `X_test` has fewer patients than `X_train`. 

Lastly, the data frame `y` contains 2 columns. The first is '`patient'`, which holds patient ids intended to correspond to the patient ids in the `X_train` and `X_test` data sets. The second is `'cancer'`, which indicates the cancer type for each patient. There are only two types: "ALL" (acute lymphoblastic leukemia) and "AML" (acute myeloid leukemia).

**Your task.** To clean the training and testing data sets, create a function `clean_data(X_train, X_test)`, which should return a new cleaned data frame `X`. The original data frames (`X_train`, `X_test`) shouldn't be changed. The cleaning process has 5 sequential steps:
* Step 1: Delete all the columns containing "call" in the column name.
* Step 2: Only keep the rows (or genes) whose `"Gene Description"` column contain the substring `"mRNA"`.
* Step 3: Delete the `"Gene Description"` column, and only keep the columns with patient ids and the `"Gene Accession Number"` column. Then, transpose the data frame, so now each column represents a gene and each row represents a patient. Use the `"Gene Accession Number"` as the new column name for each gene. Use the `'patient_id'` as the new row name for each patient. **Note: `X_test` has different patient_ids with `X_train`.**
* Step 4: Concatenate new `X_train` and `X_test data` frames into one data frame, with `X_train` above `X_test`. Then sort this combined data frame by its index in **numerical order**.
* Step 5: Convert the data type to numeric type. This last step is designed to make the later PCA and Kmeans calculations easier.  

There are 2 test cells to give you some partial credits.

In [4]:
def clean_data(X_train, X_test, y):
    # copy frames
    train = X_train.copy()
    test = X_test.copy()
    
    # take out call columns
    subset_train = [col for col in train.columns if 'call' not in col]
    subset_test = [col for col in test.columns if 'call' not in col]
    
    train = train[subset_train]
    test = test[subset_test]
    
    # filter rows by mRNA
    train = train[train['Gene Description'].str.contains("mRNA")]
    test = test[test['Gene Description'].str.contains("mRNA")]
    
    # delete gene description
    train = train.drop('Gene Description', axis=1)
    test = test.drop('Gene Description', axis=1)
    
    # transpose the data
    train = train.transpose().reset_index()
    train.columns = train.iloc[0]
    train = train.iloc[1:, :]
    
    test = test.transpose().reset_index()
    test.columns = test.iloc[0]
    test = test.iloc[1:, :]
    
    # concatenate train and test
    X = pd.concat([train, test], axis=0)
    
    # change type to numeric all columns
    X = X.apply(pd.to_numeric)
    
    # set index
    X = X.set_index('Gene Accession Number').sort_index()
    
    
    
    
    return X

In [5]:
# `clean_data_0`: Test cell (0.5 points)
# This test cell is to give you some partial credits

# Your solution for clean_data
X = clean_data(X_train, X_test, y)
assert X_train.shape == (7129, 78), "You shouldn't modify on X_train!"
assert X_test.shape == (7129, 70), "You shouldn't modify on X_test!"
assert X.shape == (72, 2037), "The shape of your X is wrong!"
print ("\n(Passed.)")


(Passed.)


In [6]:
# `clean_data_1`: Test cell (2 points)

# functions from notebook to compare tibbles
def canonicalize_tibble(X):
    var_names = sorted(X.columns)
    Y = X[var_names].copy()
    Y.sort_values(by=var_names, inplace=True)
    Y.reset_index(drop=True, inplace=True)
    return Y

def tibbles_are_equivalent(A, B):
    A_hat = canonicalize_tibble(A)
    B_hat = canonicalize_tibble(B)
    equal = (A_hat == B_hat)
    return equal.all().all()

# Loading the files into a pandas dataframe for you
X_solution = pd.read_csv(DATA_PATH + "X_solution.csv", index_col="Gene Accession Number")

# Compare your solution with teacher's solution
print("=== Last few rows of your solution ===")
display(X.tail())

print ("=== Last few rows of the instructor's solution ===")
display(X_solution.tail())

# Check it
assert tibbles_are_equivalent(canonicalize_tibble(X), canonicalize_tibble(X_solution)), \
"Please check your solution, which is different from the instructor's solution"
print ("\n(Passed.)")

=== Last few rows of your solution ===


,AB000115_at,AB000460_at,AB000464_at,AB000466_at,AB000467_at,AB000584_at,AC000099_at,AC002045_xpt1_at,AF000177_at,AF000234_at,...,L41268_f_at,X99479_f_at,S80905_f_at,Z34822_f_at,U87593_f_at,L34355_at,U48730_at,U58516_at,U73738_at,Z78285_f_at
Gene Accession Number,,,,,,,,,,,,,,,,,,,,,
68,101,1910,548,-819,-296,-154,138,166,265,50,...,244,52,-27,-161,172,-111,214,540,13,-1
69,1421,1813,348,-532,-716,-365,150,102,310,-388,...,164,113,365,-105,57,65,409,617,-34,-30
70,215,1249,371,-628,-745,80,125,57,-66,-125,...,266,216,179,-143,78,199,131,318,35,40
71,174,1300,620,-538,-976,-133,110,52,34,-245,...,117,154,210,-136,124,-106,214,760,-38,-12
72,154,1541,882,-466,-558,81,77,109,225,-254,...,231,-110,162,-142,-3,70,206,697,3,-2


=== Last few rows of the instructor's solution ===


,AB000115_at,AB000460_at,AB000464_at,AB000466_at,AB000467_at,AB000584_at,AC000099_at,AC002045_xpt1_at,AF000177_at,AF000234_at,...,L41268_f_at,X99479_f_at,S80905_f_at,Z34822_f_at,U87593_f_at,L34355_at,U48730_at,U58516_at,U73738_at,Z78285_f_at
Gene Accession Number,,,,,,,,,,,,,,,,,,,,,
68,101,1910,548,-819,-296,-154,138,166,265,50,...,244,52,-27,-161,172,-111,214,540,13,-1
69,1421,1813,348,-532,-716,-365,150,102,310,-388,...,164,113,365,-105,57,65,409,617,-34,-30
70,215,1249,371,-628,-745,80,125,57,-66,-125,...,266,216,179,-143,78,199,131,318,35,40
71,174,1300,620,-538,-976,-133,110,52,34,-245,...,117,154,210,-136,124,-106,214,760,-38,-12
72,154,1541,882,-466,-558,81,77,109,225,-254,...,231,-110,162,-142,-3,70,206,697,3,-2



(Passed.)


## Principal Components Analysis
Principal Components Analysis (PCA) is a simple yet popular and useful linear transformation technique that is used in numerous applications, such as the analysis of gene expression data. In this section, you will reimplement PCA from Notebook 15 but with a couple variations in the basic steps.

### Introduction
The main goal of a PCA analysis is to identify patterns in data; PCA aims to detect the correlation between variables. If a strong correlation between variables exists, it's likely the dimensionality can be reduced. Put differently, PCA finds the directions of maximum variance in high-dimensional data and projects the data points onto a smaller dimensional subspace while attempting to preserve most of the "information" of the data, as measured by the magnitude of the projections of the original data onto the subspace.

### PCA and Dimensionality Reduction
Often, the desired goal is to reduce the dimensions of a $d$-dimensional dataset by projecting it onto a $k$-dimensional subspace (where $k < d$). An important question is how large $k$ should be.

We will compute the eigenvectors (the principal components) of the dataset and collect them into a projection matrix. Each of those eigenvectors is associated with an eigenvalue, which can be interpreted as the "length" or "magnitude" of the corresponding eigenvector. If some eigenvalues have a significantly larger magnitude than others, then the reduction of the dataset via PCA onto a smaller dimensional subspace by dropping the "less informative" eigenpairs is reasonable.

### A Summary of the PCA Approach

In the exercises below, you will implement the following procedure.

1. Standardize the data.
2. Obtain the Eigenvectors and Eigenvalues from the covariance matrix.
3. Sort eigenvalues in descending order and choose the $k$ eigenvectors that correspond to the $k$ largest eigenvalues, where $k$ is the number of dimensions of the new feature subspace ($k \le d$).
4. Construct the projection matrix $W$ from the selected $k$ eigenvectors.
5. Transform the original dataset $X$ via $W$ to obtain a $k$-dimensional feature subspace $Y$.

### 1 - Standardizing
Whether to standardize the data prior to a PCA on the covariance matrix depends on the measurement scales of the original features. Since one interpretation of PCA is that it yields a feature subspace that maximizes the variance along the subspace's axes, it makes sense to standardize the data, especially, if it was measured on different scales. Let us continue with the transformation of the data so that it has unit scale (mean=0 and variance=1), which is a requirement for the optimal performance of many machine learning algorithms.

Recall the following concepts from basic statistics:

**Standardization (z-score)**. Let
$$
\mu = \frac{1}{n} \sum_{i=0}^{n-1} (x_i)
$$
be the mean of a collection of values $x_0, x_1, \ldots, x_{n-1}$.
Let
$$
\sigma = \sqrt{\frac{1}{(n-1)} \sum_{i=0}^{n-1} (x_i - \mu)^2}
$$
be the (**unbiased**) standard deviation of those values.
Then _standardizing the data_ (or _calculating the z-scores_) means computing for each item $x_i$ the value
$$
z_i = \frac{x_i - \mu}{\sigma}.
$$

**Exercise 1** (`Standardize`: 1 point) 

Given the data matrix `X`, return a new matrix `X_std` of the same size in which each `X_std[i, j]` element is the standardized value (z-score) with respect to the elements in the same column, `X[:, j]`. Use the definition of z-score as it appears above; in particular, be sure to compute the **unbiased** standard deviation.

In [7]:



X_std = (X - X.mean()) / X.std()


In [8]:
# `Standardization`: Test cell (1 point)
import math
sum_tol = 1e-4

print(X_std)
print(X_std.shape)

# Check it
assert X_std.shape==(72,2037), "Expected 72 rows and 2037 columns."

sum_X_std = sorted(X_std.sum(axis=1))

# Compare your solution with teacher's solution
print("=== First 2 elements and Last 2 elements of your solution ===")
print(sum_X_std[:2])
print(sum_X_std[70:])

# Check it
sum_tests = [(0,-867.0718922111674),(1,-776.1804067699011),
             (70,1078.304154730164),(71,1663.9557654929808)]

print ("=== First 2 elements and Last 2 elements of instructor's solution ===")
result_array = np.array([sum_tests[0][1],sum_tests[1][1],
                         sum_tests[2][1],sum_tests[3][1]])
print(result_array[:2])
print(result_array[2:])

# Test elements close
for i,j in sum_tests:
    assert math.isclose(sum_X_std[i],j,abs_tol=sum_tol), ("Row "+str(i)+" should sum to "+str(j)+" but yours had "+str(sum_X_std[i]))

print ("\n(Passed.)")

0                      AB000115_at  AB000460_at  AB000464_at  AB000466_at  \
Gene Accession Number                                                       
1                        -0.040869    -0.314559     0.128975     0.012511   
2                        -0.123946     0.388869     0.007705     0.208954   
3                         0.165485     0.307157     4.543723    -1.954322   
4                        -0.477695     0.539857     1.899002    -0.167163   
5                        -0.265981    -0.383836    -1.112108     1.090555   
6                        -0.603650    -0.835024    -0.007776    -1.051160   
7                        -0.343699    -0.071201    -0.887630    -1.468004   
8                        -0.000670     1.930725     0.828213    -2.155557   
9                        -0.263301     0.520317     1.914484     0.182602   
10                       -0.491094    -0.739102    -1.308205     0.949212   
11                       -0.354419     0.520317    -0.121306     0.788703   

### 2 - Eigendecomposition - Computing Eigenvectors and Eigenvalues
The eigenvectors and eigenvalues of a covariance (or correlation) matrix represent the "core" computation of a PCA. The eigenvectors are known as the _principal components_, and they determine the axes of the new feature space. The eigenvalues measure the variance of the data along the new feature axes.

#### Covariance Matrix
One method to calculate PCA is to perform the eigendecomposition on the covariance matrix $\Sigma$, which is a $d \times d$ matrix where each element $\sigma_{ij}$ is the covariance between two features $i$ and $j$. The covariance between two features is calculated as follows:
$$\sigma_{jk} = \frac{1}{n-1}\sum_{i=1}^{N}\left(  x_{ij}-\bar{x}_j \right)  \left( x_{ik}-\bar{x}_k \right)$$

We can summarize the calculation of the covariance matrix via the following matrix equation:
$$\Sigma = \frac{1}{n-1} \left( (\mathbf{X} - \mathbf{\bar{x}})^T\;(\mathbf{X} - \mathbf{\bar{x}}) \right)$$

where $\mathbf{\bar{x}}$ is the mean vector $\mathbf{\bar{x}} = \sum\limits_{k=0}^{n-1} x_{i}$.

The mean vector is a $d$-dimensional vector where each value in this vector represents the sample mean of a feature column in the dataset.

**Exercise 2** (Calculate $\Sigma$ : 1 point)

Because our data has been standardized, `X_std` is approximately equal to $\mathbf{X} - \mathbf{\bar{x}}$. Calculate a covariance matrix, `cov_mat`, using the equation for $\Sigma$ above.

In [9]:
shp = X_std.shape

cov_mat = (1/(shp[0]-1)) * (X_std.T.dot(X_std))


In [10]:
# `Covariance Matrix`: Test cell (1 point)
import math
sum_tol = 1e-4

print('Covariance matrix \n%s' %cov_mat)
print(cov_mat.shape)
print()

# Check it
assert cov_mat.shape==(2037,2037), "Expected 2037 rows and 2037 columns."

sum_cov_mat = sorted(cov_mat.sum(axis=1))

# Compare your solution with teacher's solution
print("=== First 2 elements and Last 2 elements of your solution ===")
print(sum_cov_mat[:2])
print(sum_cov_mat[2035:])

# Check it
sum_tests = [(0,-231.7563891769041),(1,-230.18940653827477),
             (2035,367.64881498520253),(2036,382.36014302238704)]

print ("=== First 2 elements and Last 2 elements of instructor's solution ===")
result_array = np.array([sum_tests[0][1],sum_tests[1][1],
                         sum_tests[2][1],sum_tests[3][1]])
print(result_array[:2])
print(result_array[2:])

# Test elements close
for i,j in sum_tests:
    assert math.isclose(sum_cov_mat[i],j,abs_tol=sum_tol), ("Row "+str(i)+" should sum to "+str(j)+" but yours had "+str(sum_cov_mat[i]))

print ("\n(Passed.)")

Covariance matrix 
0                 AB000115_at  AB000460_at  AB000464_at  AB000466_at  \
0                                                                      
AB000115_at          1.000000     0.180767     0.111231     0.091335   
AB000460_at          0.180767     1.000000     0.512389    -0.479128   
AB000464_at          0.111231     0.512389     1.000000    -0.357822   
AB000466_at          0.091335    -0.479128    -0.357822     1.000000   
AB000467_at          0.000147    -0.410275    -0.283896     0.533504   
AB000584_at         -0.079254    -0.208673    -0.224306    -0.202898   
AC000099_at          0.016718     0.347152     0.177972    -0.303668   
AC002045_xpt1_at     0.255875     0.436467     0.237483    -0.146870   
AF000177_at          0.047427     0.304363     0.131856     0.075939   
AF000234_at         -0.339367     0.065910     0.011985     0.233296   
AF000430_at         -0.012278     0.161882     0.042581    -0.119979   
AF000560_at          0.027021     0.341690   

**Exercise 3** (`Eigendecomposition on the covariance matrix`: 1 point)  

Eigenvectors and eigenvalues are numbers and vectors associated to square matrices, and together they provide the eigendecomposition of a matrix which analyzes the structure of this matrix. Even though the eigendecomposition does not exist for all square matrices, it has a particularly simple expression for a class of matrices often used in multivariate analysis such as correlation, covariance, or cross-product matrices. The eigendecomposition of this type of matrices is important in statistics because it is used to find the maximum (or minimum) of functions involving these matrices. For example, principal component analysis is obtained from the eigen-decomposition of a covariance matrix and gives the least square estimate of the original data matrix.

Calculate `eig_vals` (eigenvalues) and `eig_vecs` (eigenvectors) for `cov_mat` (the covariance matrix).

> Hint: Because this is a symmetrical array, we need to use https://docs.scipy.org/doc/numpy-1.15.0/reference/generated/numpy.linalg.eigh.html for real numbers rather than np.linalg.eig (which may find solutions using complex numbers).


In [11]:
eig_vals, eig_vecs = np.linalg.eigh(cov_mat)


In [12]:
# `Eigendecomposition`: Test cell (1 point)
import math
sum_tol = 1e-4

print('Eigenvectors \n%s' %eig_vecs)
print(eig_vecs.shape)
print('\nEigenvalues \n%s' %eig_vals)
print(eig_vals.shape)
print()

# Check it
assert eig_vecs.shape==(2037,2037), "eig_vecs: Expected 2037 rows and 2037 columns."
assert eig_vals.shape==(2037,), "eig_vals: Expected 2037 values."

sum_evec = sorted(eig_vecs.sum(axis=1))
sum_eval = sorted(eig_vals)

print("=== Sum of largest 4 elements of your eig_vals solution ===")
print("Sum:",sum_eval[-4:]," = ",sum(sum_eval[-4:]))
print("=== Sum of largest 4 elements of instructor's eig_vals solution ===")
print("Sum: [79.29171678892715, 88.80223529674713, 206.78777407920407, 309.13985245025475]  =  684.0215786151332")

assert math.isclose(sum(sum_eval[-4:]),684.0215786151332,abs_tol=sum_tol), ("Your sum is "+str(sum(sum_eval[-4:]))+" but should be ~"+str(684.0215786151332))

print ("\n(Passed.)")

Eigenvectors 
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ... -3.89372903e-03
   2.16511781e-02 -9.40331438e-03]
 [ 4.14564304e-01 -4.51291745e-01 -7.76758784e-02 ...  1.80149457e-02
   6.99866944e-03 -4.60275107e-02]
 [ 6.39314831e-01  3.06520336e-02 -1.56907802e-01 ... -6.01557472e-03
  -2.26319433e-04 -3.74154131e-02]
 ...
 [ 2.56915799e-02  1.99882009e-03 -7.62556039e-03 ... -2.22951710e-03
   3.08951885e-03 -3.37014924e-02]
 [ 3.11888921e-03  7.24722671e-03  2.18952398e-02 ... -2.75004907e-02
   1.14262773e-02  4.69653264e-03]
 [ 7.70675950e-03  4.88618868e-03  9.53699458e-03 ... -1.83383834e-02
  -7.82591261e-04  5.03114201e-04]]
(2037, 2037)

Eigenvalues 
[-8.36950640e-14 -5.16462854e-14 -3.12658162e-14 ...  8.88022353e+01
  2.06787774e+02  3.09139852e+02]
(2037,)

=== Sum of largest 4 elements of your eig_vals solution ===
Sum: [79.29171678892715, 88.80223529674716, 206.78777407920393, 309.1398524502548]  =  684.021578615133
=== Sum of largest 4 elements of instructor's e

### 3 - Selecting Principal Components
The typical goal of a PCA is to reduce the dimensionality of the original feature space by projecting it onto a smaller subspace, where the eigenvectors will form the axes. However, the eigenvectors only define the directions of the new axis, since they have all the same unit length 1, which can confirmed by the following two lines of code:

In [13]:
# Just execute this code
for ev in eig_vecs:
    np.testing.assert_array_almost_equal(1.0, np.linalg.norm(ev))
print('Everything ok!')

Everything ok!


In order to decide which eigenvector(s) can be dropped without losing too much information for the construction of lower-dimensional subspace, we need to inspect the corresponding eigenvalues. The eigenvectors with the lowest eigenvalues explain the least variance about the distribution of the data; those are the ones can be dropped.

In order to do so, the common approach is to rank the eigenvalues from highest to lowest and to choose the top $k$ eigenvectors.

In [14]:
# Just execute this code

# Make a list of (eigenvalue, eigenvector) tuples
eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i]) for i in range(len(eig_vals))]

# Sort the (eigenvalue, eigenvector) tuples from high to low
from operator import itemgetter
eig_pairs.sort(key=itemgetter(0))
eig_pairs.reverse()

# Visually confirm that the list is correctly sorted by decreasing eigenvalues
print('Eigenvalues in descending order:')
for i in eig_pairs:
    print(i[0],i[1][:3],"...")

Eigenvalues in descending order:
309.1398524502548 [-0.00940331 -0.04602751 -0.03741541] ...
206.78777407920393 [ 0.02165118  0.00699867 -0.00022632] ...
88.80223529674716 [-0.00389373  0.01801495 -0.00601557] ...
79.29171678892715 [ 0.00413807  0.00536075 -0.02178184] ...
72.32038783424807 [-0.01130794 -0.00758037  0.01329661] ...
64.98589767573822 [ 0.04330808  0.00695083 -0.00510095] ...
53.27510403805331 [-0.00461638 -0.00421056  0.02328184] ...
47.03826295799616 [ 0.01509312  0.00643272 -0.00835753] ...
46.68386257221233 [-0.0395275  -0.00094225  0.00677754] ...
43.19746764171434 [ 0.00121415 -0.00550408 -0.00810675] ...
38.018674199339294 [-1.44967951e-02 -1.38311689e-02  7.53299615e-05] ...
37.40044831341172 [ 0.05326305 -0.00392827  0.01227851] ...
32.227225531018576 [-0.02891464  0.03273987 -0.01618008] ...
31.69484347185741 [0.01295107 0.0121705  0.00785119] ...
30.54480538036044 [-0.02785237 -0.0101514  -0.02938086] ...
28.74419035074769 [0.03581549 0.00310617 0.0072434 ] ..

4.024219416069993e-15 [0.         0.00471445 0.00108354] ...
4.012302845708919e-15 [ 0.         -0.00261786  0.00160059] ...
4.008305210223018e-15 [0.         0.00476367 0.00183766] ...
4.006553591870056e-15 [ 0.00000000e+00 -6.44665798e-05 -1.01198428e-03] ...
4.004864833257109e-15 [ 0.00000000e+00 -3.95133560e-04 -9.96737229e-05] ...
4.0033808471085185e-15 [ 0.         -0.00284844 -0.00228006] ...
4.002423965208996e-15 [ 0.          0.00170264 -0.00176228] ...
3.9903762508717125e-15 [ 0.         -0.00056371  0.00026774] ...
3.9890709822023164e-15 [ 0.         -0.00124399 -0.00142797] ...
3.9889496587356795e-15 [ 0.         -0.00121812 -0.00294333] ...
3.9861342481775735e-15 [ 0.         -0.0024221   0.00221487] ...
3.972793362101215e-15 [ 0.          0.00313307 -0.00075517] ...
3.970511124019917e-15 [ 0.         -0.0032234  -0.00142689] ...
3.964223204182503e-15 [0.         0.00149695 0.00102139] ...
3.956919283049505e-15 [ 0.          0.00116975 -0.00032062] ...
3.95038088386323e-15

2.2667395653241937e-15 [ 0.         -0.00040921 -0.00212854] ...
2.2661620943196676e-15 [ 0.         -0.00055716  0.00122744] ...
2.261981503935684e-15 [ 0.          0.00010367 -0.00113471] ...
2.2618801018433214e-15 [ 0.          0.00217375 -0.00326355] ...
2.261513897840575e-15 [ 0.          0.00224748 -0.00053256] ...
2.261211552502749e-15 [ 0.         -0.0011728  -0.00048937] ...
2.2510046664819865e-15 [ 0.00000000e+00 -5.12659170e-05 -1.04578734e-05] ...
2.2460382923136084e-15 [ 0.00000000e+00 -1.85434423e-03  6.05454294e-05] ...
2.245273592265884e-15 [0.         0.00224552 0.0005964 ] ...
2.2450442413119533e-15 [ 0.          0.00226414 -0.0018858 ] ...
2.2424909715281116e-15 [ 0.00000000e+00 -6.22876031e-04 -6.21472757e-07] ...
2.2424676415042492e-15 [ 0.         -0.00210903 -0.000309  ] ...
2.239533885840149e-15 [0.         0.00351528 0.00163166] ...
2.2391372850854573e-15 [ 0.         -0.00154085  0.00144214] ...
2.2376396508258477e-15 [ 0.         -0.00063344  0.00286177] ...


9.623807733643286e-16 [ 0.         -0.0012494  -0.00191446] ...
9.61974109844877e-16 [ 0.         -0.00265514  0.00172765] ...
9.589557497972376e-16 [0.         0.00055981 0.00031277] ...
9.562964491768777e-16 [0.         0.00334445 0.00151273] ...
9.531928824319139e-16 [0.         0.00019682 0.0003967 ] ...
9.51884850281652e-16 [ 0.         -0.00074269  0.00288704] ...
9.498449358691308e-16 [ 0.          0.00357711 -0.00235515] ...
9.494272929929032e-16 [0.         0.00029875 0.0016643 ] ...
9.48129259778262e-16 [0.00000000e+00 8.34583208e-05 1.05663473e-03] ...
9.465025773962459e-16 [ 0.         -0.00101669  0.00432847] ...
9.440940900105713e-16 [ 0.00000000e+00 -1.47766959e-03 -2.75510282e-06] ...
9.438932030954928e-16 [0.         0.00054751 0.00137887] ...
9.404455421162986e-16 [ 0.         -0.00255761  0.00178658] ...
9.36208247372096e-16 [ 0.          0.00322139 -0.00337758] ...
9.324472319106648e-16 [ 0.         -0.00435574 -0.00059044] ...
9.319632739570826e-16 [ 0.          0.

**Exercise 4** (`Compute dimensionality reduction`: 2 points)  

After sorting the eigenpairs, the next question is "how many principal components should we choose for our new feature subspace?" A useful measure is the so-called "explained variance," which can be calculated from the eigenvalues. The explained variance tells us how much information (variance) can be attributed to each of the principal components.

You'll use a **75%** goal for the explained variance to choose the top $k$ components. That is, select the minimum number of the $k$-largest eigenvalues such that

$$explained\_variance = \sum_{i=0}^{k-1}\left( \frac{eigenvalue_i}{\sum_{j=0}^{N-1}\left(  eigenvalue_j \right)} \right) \geq 75\%, $$

where $k$ is the number of the top-$k$ components to choose and the covariance matrix is $N \times N$. Store the value of $k$ in a variable named `k_use` and store the explained variance from the above equation in a variable named `explained_variance`.
> Hint: See [np.cumsum](https://het.as.utexas.edu/HET/Software/Numpy/reference/generated/numpy.cumsum.html) function as used in Notebook 15. 

In [16]:
eig_total = sum(eig_vals)

explained_variance = 0
k_use = 0

for k, i in enumerate(eig_pairs):
    if explained_variance <= .75:
        ev = i[0] / eig_total
        explained_variance = explained_variance + ev
        k_use = k




In [17]:
# `select_components`: Test cell (2 points)
import math
sum_tol = 1e-4

print("Number of Components for",str(100*explained_variance),"% of variance:",k_use)

# Check it
assert k_use==29, ("Your calculated k_use is "+str(k_use)+" but should be 29.")

# Check it
assert math.isclose(explained_variance,0.7505984465835792,abs_tol=sum_tol), ("Your explained variance is "+str(explained_variance)+" but should be ~"+str(0.7505984465835792))

print ("\n(Passed.)")

Number of Components for 75.0598446583579 % of variance: 29

(Passed.)


### 4 - Construct the projection matrix $W$ from the selected $k$ eigenvectors.
It's about time to get to the really interesting part: The construction of the projection matrix that will be used to transform the gene expression data onto the new feature subspace. The "projection matrix" is nothing more than a matrix whose columns are our top $k$ eigenvectors.

The code cell below computes the $d \times k$ projection matrix $W$.

In [18]:
num_features = X.shape[1]

p = np.array(eig_pairs[0][1].reshape(num_features,1))

for i in range(1,k_use):
    p = np.append(p, eig_pairs[i][1].reshape(num_features,1), 0)

p=p.reshape(num_features,k_use)

matrix_w = np.hstack(np.vsplit(p,1))

print('Matrix W:\n', matrix_w)
print(matrix_w.shape)

Matrix W:
 [[-0.00940331 -0.04602751 -0.03741541 ... -0.01512247 -0.02157882
   0.02455123]
 [-0.01303358 -0.00035806 -0.00681776 ...  0.00824519 -0.03727329
   0.02460511]
 [-0.0002826  -0.00933614 -0.04544453 ... -0.02125062 -0.01634545
   0.00613641]
 ...
 [ 0.01198928  0.01553509  0.00416808 ... -0.01282362 -0.04878263
  -0.0058803 ]
 [ 0.01483216 -0.00776181 -0.01263376 ... -0.03376636 -0.0237215
  -0.03499303]
 [-0.01606375  0.0044885   0.00433831 ... -0.01457857  0.00349194
  -0.00887598]]
(2037, 29)


### 5 - Projection Onto the New Feature Space
In this last step we will use the $2037×k$-dimensional projection matrix $W$ to transform our samples onto the new subspace via the equation

$Y=X×W$, where $Y$ is a $72×k$ matrix of our transformed samples.

In [19]:
Y_calc = X_std.dot(matrix_w)

print(Y_calc)
print(Y_calc.shape)

                             0         1         2         3         4   \
Gene Accession Number                                                     
1                     -1.004462 -1.957466 -0.337312 -0.493736 -1.007024   
2                      1.300662 -0.336089 -0.189284 -0.519404  0.629756   
3                      1.597998 -0.631023 -3.327653  0.248270  1.245970   
4                      0.559081  0.026557 -2.099599 -0.202852 -0.484994   
5                      0.215966  1.302240  0.251360  0.126948 -1.645542   
6                     -0.522915  0.156602 -0.348410  0.183875  0.595347   
7                      0.264036 -0.347724 -0.461893 -1.185948  0.263115   
8                     -1.198207 -0.179004 -2.086847  0.232710  1.697997   
9                      1.840883 -0.864249 -1.389545 -0.247218  0.938665   
10                    -0.178288  0.077262  0.259325 -0.032251 -1.158089   
11                     1.286581  0.931675 -0.134079  0.261472 -0.224650   
12                     1.

## Shortcut - PCA in scikit-learn
For educational purposes, we went a long way to apply the PCA to the Gene Expression dataset. But luckily, there is already implementation in scikit-learn.

In [20]:
from sklearn.decomposition import PCA as sklearnPCA
sklearn_pca = sklearnPCA(n_components=k_use)
Y_sklearn = sklearn_pca.fit_transform(X_std)

print(Y_sklearn)
print(Y_sklearn.shape)

[[ 15.37761032   3.22022452   0.96875489 ...   0.27815135   7.49733916
    0.5367048 ]
 [  3.55628996   7.87850935 -11.50161375 ...   7.79210781  -5.9569517
   -2.44920953]
 [ 33.50974262   2.34889207  -5.9811807  ...  -2.27917817   7.01825224
   14.09741499]
 ...
 [-24.60252152  -7.28972663   0.701094   ...  -3.08214559   2.97111965
    1.01832411]
 [-11.59790658   2.60313947  -4.21848613 ...  -1.29541514   2.17737695
   -4.44590187]
 [ -1.96149321   9.01805144  -1.34471463 ...  -0.93251268   0.88258935
   -8.19226909]]
(72, 29)


**Note:** Any differences between `Y_calc` & `Y_sklearn` may be ignored and are due to the way the data was standardized for the PCA algorithm exercises.

## Kmeans Clustering
**Exercise 5** (`Kmeans_accuracy`: 2.5 points)


Create a function `kmeans_accuracy(X_std, initial_index, y_labels)`, which using kmeans clustering to cluster the dataset X into K clusters. `K = len(initial_index)`. The output should be the accuarcy of clustering comparing to the `y_labels`. To make it easier, we will tell you to write this function in 2 steps. To give you some partial credits, we split the test into 2 cells. One is to check your accuracy on standardized original data set `X_std`, which has 1.5 points. The other is to check the accuracy on PCA reconstrcuted data set `Y_calc`, which has 1 point. 
* **Step 1**: Performing kmeans clustering on standardized data. Given the `initial_index` as a list of pre-selected patient_id, generate the intial_centers from X_std, and perform kmeans clustering, return the clustering labels for all the patients.
* **Step 2**: Comparing the accuracy between your clustering labels and the `y_labels` (i.e. `y['cancer']`). **Note**: The clustering label 0 may or may not be associated with patient type "ALL", you need to figure out which cluster label is associated with which patient type. 
>Hint: You can use functions from Notebook 14, which are included in the cell below. 


**Notes:**
1. You need to use the specified index to generate the initial centroids, instead of just using cluster number K. The inital_index is given in the test cell.
2. You can use any functions in notebook 14 (all needed functions are provided in the cell below) **OR** create your own functions, **OR** using the vq module from scipy.cluster package (You can use either kmeans or kmeans2 functions from scipy.cluster.vq). 

In [23]:
## function from notebook 14
def compute_d2(X, centers):
    """Computing the distance matrix S."""
    m = len(X)
    k = len(centers)
    S = np.empty((m, k))
    for i in range(m):
        d_i = np.linalg.norm(X[i, :] - centers, ord=2, axis=1)
        S[i, :] = d_i**2
    return S
  
def assign_cluster_labels(S):
    return np.argmin(S, axis=1)

def update_centers(X, y):
    # X[:m, :d] == m points, each of dimension d
    # y[:m] == cluster labels
    m, d = X.shape
    k = max(y) + 1
    assert m == len(y)
    assert (min(y) >= 0)
    
    centers = np.empty((k, d))
    for j in range(k):
        # Compute the new center of cluster j,
        # i.e., centers[j, :d].
        centers[j, :d] = np.mean(X[y == j, :], axis=0)
    return centers

def WCSS(S):
    return np.sum(np.amin(S, axis=1))
  
def has_converged(old_centers, centers):
    return set([tuple(x) for x in old_centers]) == set([tuple(x) for x in centers])
  
def kmeans(X, starting_centers, max_steps=np.inf):
    centers = starting_centers
    converged = False
    labels = np.zeros(len(X))
    i = 1
    while (not converged) and (i <= max_steps):
        old_centers = centers
        S = compute_d2(X, centers)
        labels = assign_cluster_labels(S)
        centers = update_centers(X, labels)
        converged = has_converged(old_centers, centers)
        #print ("iteration", i, "WCSS = ", WCSS (S))
        i += 1
    return labels



def mark_matches(a, b, exact=False):
    """
    Given two Numpy arrays of {0, 1} labels, returns a new boolean
    array indicating at which locations the input arrays have the
    same label (i.e., the corresponding entry is True).
    
    This function can consider "inexact" matches. That is, if `exact`
    is False, then the function will assume the {0, 1} labels may be
    regarded as the same up to a swapping of the labels. This feature
    allows
    
      a == [0, 0, 1, 1, 0, 1, 1]
      b == [1, 1, 0, 0, 1, 0, 0]
      
    to be regarded as equal. (That is, use `exact=False` when you
    only care about "relative" labeling.)
    """
    assert a.shape == b.shape
    a_int = a.astype(dtype=int)
    b_int = b.astype(dtype=int)
    all_axes = tuple(range(len(a.shape)))
    assert ((a_int == 0) | (a_int == 1)).all()
    assert ((b_int == 0) | (b_int == 1)).all()
    
    exact_matches = (a_int == b_int)
    if exact:
        return exact_matches

    assert exact == False
    num_exact_matches = np.sum(exact_matches)
    if (2*num_exact_matches) >= np.prod (a.shape):
        return exact_matches
    return exact_matches == False # Invert
    
def count_matches(a, b, exact=False):
    """
    Given two sets of {0, 1} labels, returns the number of mismatches.
    
    This function can consider "inexact" matches. That is, if `exact`
    is False, then the function will assume the {0, 1} labels may be
    regarded as similar up to a swapping of the labels. This feature
    allows
    
      a == [0, 0, 1, 1, 0, 1, 1]
      b == [1, 1, 0, 0, 1, 0, 0]
      
    to be regarded as equal. (That is, use `exact=False` when you
    only care about "relative" labeling.)
    """
    matches = mark_matches(a, b, exact=exact)
    return np.sum(matches)

In [41]:
from scipy.cluster import vq

def kmeans_accuracy(X, initial_index, y_labels):
    
    X = X.copy()
    
    first = np.asarray(X[X.index == initial_index[0]])
    second = np.asarray(X[X.index == initial_index[1]])
    my_array = np.column_stack((first[0], second[0])).T
    
    centers_vq, distortion_vq = vq.kmeans(X, k_or_guess=my_array)
    clustering_vq, _ = vq.vq(X, centers_vq)
    clustering = pd.DataFrame(clustering_vq)
    clustering['labels'] = y_labels
    clustering['labels'] = np.where(clustering.labels == 'ALL', 1, 0)
    clustering.columns = ['kmeans', 'labels']
    n_matches_vq = count_matches(clustering['labels'], clustering['kmeans'], exact=True)
    
    
    return n_matches_vq / len(clustering)


initial_index = [30, 50]

# Note: this is not the solution for exercise 1. We made them different intendedly.
Y_std = (Y_calc/np.std(Y_calc, axis=0))

kmeans_accuracy(Y_std, initial_index, y['cancer'])




0.6666666666666666

In [42]:
# `kmeans_accuracy_test0`: Test cell 0 (1.5 points)
# initial index
initial_index = [30, 50]

%timeit acc_original = kmeans_accuracy(X_std, initial_index, y['cancer'])
acc_original = kmeans_accuracy(X_std, initial_index, y['cancer'])

sum_tol = 1e-4
assert (acc_original - 0.6944444444444444) <= sum_tol, "Your clustering accuracy on original data set is wrong!"

print ("\n(Passed.)")

6.55 ms ± 98.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

(Passed.)


In [43]:
# `kmeans_accuracy_test1`: Test cell 1 (1 points)
# initial index
initial_index = [30, 50]

# Note: this is not the solution for exercise 1. We made them different intendedly.
Y_std = (Y_calc/np.std(Y_calc, axis=0))

%timeit acc_pca = kmeans_accuracy(Y_std, initial_index, y['cancer'])
acc_pca = kmeans_accuracy(Y_std, initial_index, y['cancer'])

sum_tol = 1e-4
assert (acc_pca - 0.625) <= sum_tol, "Your clustering accuracy on pca data set is wrong!"

print ("\n(Passed.)")

3.75 ms ± 6.34 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


AssertionError: Your clustering accuracy on pca data set is wrong!

Congratulations! Now you have implemented kmeans_accuracy on both original and PCA reconstructed gene expression data set. Do you have a conclusion which method is better? It's hard to tell on this small data set. But imaging you have more genes and more patients, then performing PCA will be faster and with similar accuracy.  

## Visualizing the results of PCA

> The remaining code cells below are optional. There is no code to write, so you can just run the cells to verify that they complete and submit the notebook. Later, when you come back to this material, you may wish to study or use the cells below as examples of different visualization techniques.

The analysis reveals that `k_use` principle components are needed to account for 75% of the variance. PC 1-3 add up to about ~30% and the rest is a slow burn where each component after PC10 contributes between 1-2% of the variance. 1% is a decent amonut of variance and so the number of important PCs is up for interpretation.

In [ ]:
"""
from bokeh.io import output_notebook, show
output_notebook()
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.plotting import figure
from bokeh.models.ranges import Range1d

# make a dataframe
xlabels = ['PC{}'.format(i) for i in range(1, k_use+2)]
var_exp = [(i / sum(eig_vals))*100 for i in sorted(eig_vals, reverse=True)]
cum_var_exp = np.cumsum(var_exp)
df = dict(x=xlabels, y=cum_var_exp[:k_use+1], h=var_exp[:k_use+1])
df = pd.DataFrame.from_dict(df)
source = ColumnDataSource(df)
# create the figure size and title
p = figure(plot_width=700, plot_height=300, title="Importance of Principal Components", x_range=xlabels)
# don't show horizontal and vertical grid line
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
p.xaxis.axis_label = "Principal Components"
p.xaxis.major_label_orientation = 1.2   # turn the x-label with an angle
# bar plot for explained variance
p.vbar(x='x', top='h', width=0.8, source=source,
       line_color="white", fill_color="dodgerblue", legend="Explained Variance",
       hover_line_color="darkgrey", hover_fill_color="dodgerblue")
# add a line glyph object which references columns from source data
p.line(x='x', y='y', legend="Cumulative Variance", source=source, line_width=2, line_color="orange")
# Setting the y  axis range   
p.y_range = Range1d(0, 80)

# set the legend location
p.legend.location = "top_left"
# show values on the plot when mouse present
p.add_tools(HoverTool(tooltips=[("Cumulative Variance", "@y"), ("Explained Variance", "@h")]))

show(p)
"""

### Projection of first three components¶
The first three components only explain 30% of the variance but we'll go ahead plot the projection to get a visual of it. In the following Markdown, we showed the codes of how to plot the figure. But since we don't have a plotly account to create unlimited figures for every student, we just insert the image instead of running the codes. You can run the codes by yourself with your own plotly username and key. Then you can twist the figure, zoom in and zoom out and check the value for each point.

In [ ]:
#Image(filename="plotly.PNG")

In [ ]:
#Image(filename="projection.png")

**Fin!** You've reached the end of this problem. Don't forget to restart the
kernel and run the entire notebook from top-to-bottom to make sure you did
everything correctly. If that is working, try submitting this problem. (Recall
that you *must* submit and pass the autograder to get credit for your work!)